In [1]:
!pip install --quiet pylance duckdb

In [2]:
import lance
import duckdb
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset
import shutil

## Converting from other formats

Via pyarrow it's really easy to create lance datasets

create a parquet dataset

In [3]:
shutil.rmtree("/tmp/test.lance", ignore_errors=True)
shutil.rmtree("/tmp/test.parquet", ignore_errors=True)

df = pd.DataFrame({"a": [5]})
tbl = pa.Table.from_pandas(df)
pa.dataset.write_dataset(tbl, "/tmp/test.parquet", format='parquet')

parquet = pa.dataset.dataset("/tmp/test.parquet")
parquet.to_table().to_pandas()

,a
0,5


Convert it to lance in 1 line of code

In [4]:
dataset = lance.write_dataset(parquet, "/tmp/test.lance")

In [5]:
# make sure it's the same
dataset.to_table().to_pandas()

,a
0,5


## Versioning

We can append rows

In [6]:
df = pd.DataFrame({"a": [10]})
tbl = pa.Table.from_pandas(df)
dataset = lance.write_dataset(tbl, "/tmp/test.lance", mode="append")

dataset.to_table().to_pandas()

,a
0,5
1,10


We can overwrite the data and create a new version

In [7]:
df = pd.DataFrame({"a": [50, 100]})
tbl = pa.Table.from_pandas(df)
dataset = lance.write_dataset(tbl, "/tmp/test.lance", mode="overwrite")

In [8]:
dataset.to_table().to_pandas()

,a
0,50
1,100


The old version is still there

In [9]:
dataset.versions()

[{'version': 2,
  'timestamp': datetime.datetime(2023, 2, 9, 21, 6, 22),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2023, 2, 9, 21, 6, 22),
  'metadata': {}},
 {'version': 1,
  'timestamp': datetime.datetime(2023, 2, 9, 21, 6, 22),
  'metadata': {}}]

In [10]:
lance.dataset('/tmp/test.lance', version=1).to_table().to_pandas()

,a
0,5


In [11]:
lance.dataset('/tmp/test.lance', version=2).to_table().to_pandas()

,a
0,5
1,10


## Vectors

### Data preparation

For this tutorial let's use the Sift 1M dataset:

- Download `ANN_SIFT1M` from: http://corpus-texmex.irisa.fr/
- Direct link should be `ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz`
- Download and then unzip the tarball

In [12]:
!rm -rf sift* vec_data.lance
!wget ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
!tar -xzf sift.tar.gz

--2023-02-09 21:06:22--  ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
           => ‘sift.tar.gz’
Resolving ftp.irisa.fr (ftp.irisa.fr)... 131.254.254.45, 2001:660:7303:254::45
Connecting to ftp.irisa.fr (ftp.irisa.fr)|131.254.254.45|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /local/texmex/corpus ... done.
==> SIZE sift.tar.gz ... 168280445
==> PASV ... done.    ==> RETR sift.tar.gz ... done.
Length: 168280445 (160M) (unauthoritative)

sift.tar.gz         100%[===================>] 160.48M  7.82MB/s    in 21s     

2023-02-09 21:06:46 (7.59 MB/s) - ‘sift.tar.gz’ saved [168280445]



Convert it to Lance

In [13]:
import struct

uri = "vec_data.lance"

with open("sift/sift_base.fvecs", mode="rb") as fobj:
    buf = fobj.read()
    data = np.array(struct.unpack("<128000000f", buf[4 : 4 + 4 * 1000000 * 128]))

    schema = pa.schema([
        pa.field("id", pa.uint32(), False),
        pa.field("vector", pa.list_(pa.float32(), 128), False)
    ])
    table = pa.Table.from_arrays([
        pa.array(range(1000000), type=pa.uint32()),
        pa.FixedSizeListArray.from_arrays(pa.array(data, type=pa.float32()), list_size=128)
    ], schema=schema)

    lance.write_dataset(table, uri, max_rows_per_group=8192, max_rows_per_file=1024*1024)

In [14]:
uri = "vec_data.lance"
sift1m = lance.dataset(uri)

### KNN (no index)

Sample 100 vectors as query vectors

In [15]:
import duckdb
vtable = sift1m.to_table() # Next release of DuckDB would no longer require this
samples = duckdb.query("SELECT vector FROM vtable USING SAMPLE 100").to_df().vector
samples

0     [19.0, 14.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0....
1     [43.0, 24.0, 3.0, 116.0, 0.0, 0.0, 0.0, 0.0, 0...
2     [0.0, 1.0, 0.0, 12.0, 144.0, 3.0, 0.0, 0.0, 0....
3     [6.0, 11.0, 3.0, 29.0, 3.0, 0.0, 0.0, 0.0, 0.0...
4     [14.0, 1.793662034335766e-43, 10.0, 2.0, 1.0, ...
                            ...                        
95    [8.0, 0.0, 0.0, 144.0, 50.0, 0.0, 0.0, 0.0, 0....
96    [15.0, 2.0, 0.0, 11.0, 140.0, 6.0, 0.0, 0.0, 0...
97    [3.0, 0.0, 0.0, 0.0, 1.0, 5.0, 34.0, 136.0, 11...
98    [131.0, 5.0, 3.0, 9.0, 17.0, 4.0, 7.0, 34.0, 7...
99    [0.0, 0.0, 0.0, 4.0, 93.0, 40.0, 74.0, 16.0, 3...
Name: vector, Length: 100, dtype: object

Call nearest neighbors (no ANN index here)

In [16]:
import time

start = time.time()
tbl = sift1m.to_table(columns=["id"], nearest={"column": "vector", "q": samples[0], "k": 10})
end = time.time()

print(f"Time(sec): {end-start}")
print(tbl.to_pandas())

Time(sec): 0.06343197822570801
       id                                             vector    score
0  247652  [19.0, 14.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0....      0.0
1  613109  [3.0, 3.0, 1.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...  29151.0
2  762749  [12.0, 2.0, 1.0, 9.0, 9.0, 0.0, 0.0, 0.0, 0.0,...  30637.0
3  907874  [0.0, 9.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0...  31257.0
4  679673  [13.0, 11.0, 7.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0....  31480.0
5  644327  [0.0, 26.0, 35.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0....  35835.0
6  843632  [8.0, 27.0, 15.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0....  36150.0
7  970697  [5.0, 5.0, 8.0, 19.0, 1.0, 0.0, 0.0, 0.0, 1.0,...  36622.0
8  539063  [0.0, 0.0, 1.0, 25.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  37416.0
9  678125  [11.0, 1.0, 11.0, 24.0, 0.0, 6.0, 2.0, 0.0, 0....  41063.0


Without the index this is scanning through the whole dataset to compute the distance. <br/>

For real-time serving we can do much better with an ANN index

### Build index

Now let's build an index. We haven't implemented HNSW but IVF+PQ is shown here

**NOTE** If you'd rather not wait for index build, you can download a version with the index pre-built from [here](https://eto-public.s3.us-west-2.amazonaws.com/datasets/sift/sift_ivf256_pq16.tar.gz) and skip the next cell

In [17]:
sift1m.create_index("vector",
                    index_type="IVF_PQ", 
                    num_partitions=256,  # IVF
                    num_sub_vectors=16)  # PQ

Building vector index: IVF256,PQ16
Sample 65536 out of 1000000 to train kmeans of 128 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 10000

### Try nearest neighbors again with ANN index

Let's look for nearest neighbors again

In [18]:
sift1m = lance.dataset(uri)

In [19]:
import time

tot = 0
for q in samples:
    start = time.time()
    tbl = sift1m.to_table(nearest={"column": "vector", "q": q, "k": 10})
    end = time.time()
    tot += (end - start)

print(f"Avg(sec): {tot / len(samples)}")
print(tbl.to_pandas())

Avg(sec): 0.0006328439712524414
       id                                             vector         score
0  343935  [0.0, 0.0, 0.0, 4.0, 93.0, 40.0, 74.0, 16.0, 3...  21943.330078
1  308331  [0.0, 0.0, 13.0, 54.0, 1.0, 28.0, 127.0, 34.0,...  77403.796875
2  964812  [17.0, 15.0, 8.0, 4.0, 24.0, 73.0, 39.0, 4.0, ...  79077.609375
3  142566  [0.0, 0.0, 0.0, 13.0, 86.0, 53.0, 34.0, 24.0, ...  80969.914062
4  681141  [10.0, 2.0, 0.0, 0.0, 71.0, 12.0, 14.0, 69.0, ...  84327.671875
5  825105  [0.0, 0.0, 0.0, 14.0, 77.0, 78.0, 111.0, 85.0,...  85784.445312
6  205260  [9.0, 22.0, 71.0, 12.0, 89.0, 99.0, 33.0, 19.0...  88762.515625
7  858258  [1.0, 0.0, 0.0, 0.0, 1.0, 30.0, 79.0, 29.0, 10...  89653.968750
8  649536  [3.0, 8.0, 19.0, 0.0, 0.0, 2.0, 55.0, 12.0, 7....  91701.023438
9  956814  [111.0, 0.0, 0.0, 3.0, 115.0, 49.0, 91.0, 44.0...  92668.718750


The latency vs recall is tunable via:
- nprobes: how many IVF partitions to search
- refine_factor: determines how many vectors are retrieved during re-ranking

In [20]:
%%time

sift1m.to_table(nearest={"column": "vector", 
                         "q": samples[0], 
                         "k": 10, 
                         "nprobes": 10, 
                         "refine_factor": 5}).to_pandas()

CPU times: user 2.54 ms, sys: 2.44 ms, total: 4.99 ms
Wall time: 2.83 ms


,id,vector,score
0,247652,"[19.0, 14.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0....",0.0
1,613109,"[3.0, 3.0, 1.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",29151.0
2,762749,"[12.0, 2.0, 1.0, 9.0, 9.0, 0.0, 0.0, 0.0, 0.0,...",30637.0
3,907874,"[0.0, 9.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0...",31257.0
4,679673,"[13.0, 11.0, 7.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0....",31480.0
5,644327,"[0.0, 26.0, 35.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0....",35835.0
6,843632,"[8.0, 27.0, 15.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0....",36150.0
7,970697,"[5.0, 5.0, 8.0, 19.0, 1.0, 0.0, 0.0, 0.0, 1.0,...",36622.0
8,539063,"[0.0, 0.0, 1.0, 25.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",37416.0
9,678125,"[11.0, 1.0, 11.0, 24.0, 0.0, 6.0, 2.0, 0.0, 0....",41063.0


q => sample vector

k => how many neighbors to return

nprobes => how many partitions (in the coarse quantizer) to probe

refine_factor => controls "re-ranking". If k=10 and refine_factor=5 then retrieve 50 nearest neighbors by ANN and re-sort using actual distances then return top 10. This improves recall without sacrificing performance too much

**NOTE** the latencies above include file io as lance currently doesn't hold anything in memory. Along with index building speed, creating a purely in memory version of the dataset would make the biggest impact on performance.

## Features and vector can be retrieved together

Usually we have other feature or metadata columns that need to be stored and fetched together.
If you're managing data and the index separately, you to do a bunch of annoying plumbing to put stuff together. With Lance it's a single call

In [21]:
tbl = sift1m.to_table()
tbl = tbl.append_column("item_id", pa.array(range(len(tbl))))
tbl = tbl.append_column("revenue", pa.array((np.random.randn(len(tbl))+5)*1000))
tbl.to_pandas()

,id,vector,item_id,revenue
0,0,"[0.0, 16.0, 35.0, 5.0, 32.0, 31.0, 14.0, 10.0,...",0,4961.761499
1,1,"[1.8e-43, 14.0, 35.0, 19.0, 20.0, 3.0, 1.0, 13...",1,5367.481158
2,2,"[33.0, 1.8e-43, 0.0, 1.0, 5.0, 3.0, 44.0, 40.0...",2,5492.358178
3,3,"[23.0, 10.0, 1.8e-43, 12.0, 47.0, 14.0, 25.0, ...",3,5751.117211
4,4,"[27.0, 29.0, 21.0, 1.8e-43, 1.0, 1.0, 0.0, 0.0...",4,4758.933137
...,...,...,...,...
999995,999995,"[8.0, 9.0, 5.0, 0.0, 10.0, 39.0, 72.0, 68.0, 3...",999995,4645.884921
999996,999996,"[3.0, 28.0, 55.0, 29.0, 35.0, 12.0, 1.0, 2.0, ...",999996,4074.818920
999997,999997,"[0.0, 13.0, 41.0, 72.0, 40.0, 9.0, 0.0, 0.0, 0...",999997,4442.169660
999998,999998,"[41.0, 121.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24...",999998,5203.739183


In [22]:
sift1m = lance.write_dataset(tbl, uri, mode="overwrite")

In [23]:
sift1m.to_table(columns=["revenue"], nearest={"column": "vector", "q": samples[0], "k": 10}).to_pandas()

,revenue,vector,score
0,4092.347205,"[19.0, 14.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0....",0.0
1,6546.615173,"[3.0, 3.0, 1.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",29151.0
2,5479.443366,"[12.0, 2.0, 1.0, 9.0, 9.0, 0.0, 0.0, 0.0, 0.0,...",30637.0
3,3915.412557,"[0.0, 9.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0...",31257.0
4,1933.213882,"[13.0, 11.0, 7.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0....",31480.0
5,5142.426668,"[0.0, 26.0, 35.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0....",35835.0
6,4276.011840,"[8.0, 27.0, 15.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0....",36150.0
7,6832.484143,"[5.0, 5.0, 8.0, 19.0, 1.0, 0.0, 0.0, 0.0, 1.0,...",36622.0
8,4393.929300,"[0.0, 0.0, 1.0, 25.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",37416.0
9,2989.297897,"[11.0, 1.0, 11.0, 24.0, 0.0, 6.0, 2.0, 0.0, 0....",41063.0
